<a href="https://colab.research.google.com/github/skolix15/Machine_Learning_2025/blob/main/Exercise_six_(6).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries

In [ ]:
import sys
!{sys.executable} -m pip install -U ydata-profiling[notebook]
!pip install jupyter-contrib-nbextensions
!jupyter nbextension enable --py widgetsnbextension

In [24]:
import pandas as pd
from ydata_profiling import ProfileReport
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Question 1

In [7]:
# Load Data
df = pd.read_csv("bankloan.csv")
# Create profile report object
profile = ProfileReport(df, title="Pandas Profiling Report", explorative=True)
# # Appear to notebook
# profile.to_notebook_iframe()
# Save to HTML file
profile.to_file("notebook_report.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]


100%|██████████| 53/53 [00:15<00:00,  3.40it/s]
/usr/local/lib/python3.12/dist-packages/pandas/core/nanops.py:1632: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  return spearmanr(a, b)[0]
/usr/local/lib/python3.12/dist-packages/pandas/core/nanops.py:1632: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  return spearmanr(a, b)[0]
/usr/local/lib/python3.12/dist-packages/pandas/core/nanops.py:1632: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  return spearmanr(a, b)[0]
/usr/local/lib/python3.12/dist-packages/pandas/core/nanops.py:1632: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  return spearmanr(a, b)[0]
/usr/local/lib/python3.12/dist-packages/pandas/core/nanops.py:1632: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  return spearmanr(a, b)[0]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

# Question 2

## Subquestion 1

Mean: 15116.256  

Max: 35000

Min: 1000

## Subquestion 2

Μπορούμε να αφαιρέσουμε τις παρακάτω μεταβλητές:

(a) Μεταβλητές που λειτουργούν ως αναγνωριστικά =>
    1. id (μοναδικές τιμές),
    2. mebmer_id (μοναδικές τιμές),
    3. row_id (μοναδικές τιμές),
    4. unnamed (είναι μη υποστηριζόμενου τύπου και έχει 100% ελλειπούσες τιμές)

(b) Μεταβλητές με μεγάλο ποσοστό ελλειπούσων τιμώ =>
    5. annual_inc_joint (99.9% ελλειπούσες τιμές)
    6. dti_joint (99.9% ελλειπούσες τιμές)
    7. 36months (86.1% ελλειπούσες τιμές, είναι υψηλά συσχετισμένη με τις μεταβλητές 60months & term)
    8. 60months (86.1% ελλειπούσες τιμές, είναι υψηλά συσχετισμένη με τις μεταβλητές 36months & term)
    9. next_pymnt_d (75.5% ελλειπούσες τιμές και μη ισρροπημένο -> 98.8%)
    10. mths_since_last_major_derog (71.8% ελλειπούσες τιμές)

(c) Μεταβλητές με υψηλή συσχέτιση =>
    11. title (υψηλά συσχετισμένη με τη μεταβλητή purpose, μπορεί να αφαιρεθεί διατηρώντας την purpose)
    12. grade & sub_grade (υψηλά συσχετισμένες μεταξύ τους. Ενδεχομένως, να αρκούσε η μία από τις δύο)
    13. funded_amnt & loan_amnt (υψηλά συσχετισμένες μεταξύ τους. Ενδεχομένως, να αρκούσε η μία από τις δύο)

In [8]:
# Set variable to drop
variables_to_drop = [

  # Identifiers/Non-predictive features
  'id', 'member_id', 'Row ID', 'Unnamed: 50',

  # Variables with high percentage of missing values (>70%)
  'annual_inc_joint', 'dti_joint', '36months', '60months',
  'next_pymnt_d', 'mths_since_last_major_derog', # ~71.8% missing

  # Redundant/Highly correlated variables
  'title', 'grade', 'funded_amnt'

]

# Drop the specified columns from the DataFrame
df_cleaned = df.drop(columns=variables_to_drop, axis=1)

# Print information
print(f"\nVariables dropped: {variables_to_drop}")
print(f"Cleaned DataFrame dimensions: {df_cleaned.shape}")


Variables dropped: ['id', 'member_id', 'Row ID', 'Unnamed: 50', 'annual_inc_joint', 'dti_joint', '36months', '60months', 'next_pymnt_d', 'mths_since_last_major_derog', 'title', 'grade', 'funded_amnt']
Cleaned DataFrame dimensions: (96184, 40)


## Subquestion 3

In [9]:
# Fill undefined/null values

# Get numerical columns
numerical_columns = df_cleaned.select_dtypes(include=['number']).columns

# Get categorical columns
categorical_columns = df_cleaned.select_dtypes(include=['object']).columns

# Fill empty numerical columns with mean value
for column in numerical_columns:
  if df_cleaned[column].isnull().any():
          mean_value = df_cleaned[column].mean()
          df_cleaned[column].fillna(mean_value)

# Fill empty categorical columns with most used value (mode)
for column in categorical_columns:
    if df_cleaned[column].isnull().any():
        # Using .iloc[0] in order to get the first value, in case of multiple modes
        mode_value = df_cleaned[column].mode().iloc[0]
        df_cleaned[column].fillna(mode_value)

# Store updated/cleaned dataframe in new csv file
df_cleaned.to_csv("bankloan_cleaned.csv", index=False)
print("Updated/cleaned Dataframe was stored in 'bankloan_cleaned.csv' file!")

Updated/cleaned Dataframe was stored in 'bankloan_cleaned.csv' file!


## Subquestion 4

In [15]:
# Set target values/subgrades
target_sub_grades = ['A1', 'A2', 'A3', 'A4', 'A5', 'B1', 'B2']

# Create target column with values 0 and 1
# Target: 1 => if sub_grade is in target_sub_grades
# Target: 0 =>  otherwise
df_cleaned['target_loan_risk'] = df_cleaned['sub_grade'].apply(lambda x: 1 if x in target_sub_grades else 0)

# Calculate target counts & perchentages
target_counts = df_cleaned['target_loan_risk'].value_counts()
target_percentages = (df_cleaned['target_loan_risk'].value_counts(normalize=True) * 100).round(2).astype(str) + ' %'

# Print results
print(f"Target Counts:\n{target_counts.to_string(header=False)}\n")
print(f"Target Percentages:\n{target_percentages.to_string(header=False)}\n")

Target Counts:
0    68675
1    27509

Target Percentages:
0    71.4 %
1    28.6 %



Με βάση τα παραπάνω στατιστικά, παρατηρούμε ότι η μεταβλητή target παρουσιάζει σημαντική ανισορροπία. Η κλάση 0 (Υψηλότερος Κίνδυνος/Non-Target) κυριαρχεί με $71.32\%$, ενώ η κλάση 1 (Χαμηλός Κίνδυνος/Target) αποτελεί τη μειοψηφία ($28.68\%$).

## Subquestion 5

In [22]:
# Create ranges (bins) of loan amount, every 2000 units
# Range begins from minimum loan_amnt and ends to a unit a little bit more than the maximum loan_amnt
bins = range(
    int(df_cleaned['loan_amnt'].min()),
    int(df_cleaned['loan_amnt'].max()) + 2000,
    2000
)

# Separate each amnt value to the respective bin and creating a new column named "loan_bin"
df_cleaned['loan_bin'] = pd.cut(df_cleaned['loan_amnt'], bins)

# Group data per bin and calculate mean of target_loan_risk
# Mean here shows the perchentage of the applications in each bin that are approved
approval_rate = df_cleaned.groupby("loan_bin", observed=True)['target_loan_risk'].mean()

# Filter bins that have accept perchentage >=15%
desired_bins = approval_rate[approval_rate >= 0.15]

# Print results
print(f"Desired ranges:\n\n{desired_bins}\n")



Desired ranges:

loan_bin
(1000, 3000]      0.180332
(3000, 5000]      0.285393
(5000, 7000]      0.346247
(7000, 9000]      0.344271
(9000, 11000]     0.331146
(11000, 13000]    0.289496
(13000, 15000]    0.297250
(15000, 17000]    0.267769
(17000, 19000]    0.266010
(19000, 21000]    0.287737
(21000, 23000]    0.218659
(23000, 25000]    0.320585
(25000, 27000]    0.229575
(27000, 29000]    0.399450
(29000, 31000]    0.163382
Name: target_loan_risk, dtype: float64



# Question 3

## Subquestiion 1

Θα επιλέξω να εφαρμόσω το StandardScaler για την κανονικοποίηση των αριθμητικών χαρακτηριστικών, καθώς οι μεταβλητές του dataset διαφέρουν σημαντικά σε κλίμακα και μονάδες μέτρησης. Η χρήση του StandardScaler διασφαλίζει ότι όλα τα χαρακτηριστικά έχουν μέση τιμή 0 και τυπική απόκλιση 1, καθιστώντας τα συγκρίσιμα μεταξύ τους. Αυτό είναι σημαντικό για μοντέλα όπως η Logistic Regression, το SVM ή το KNN, τα οποία επηρεάζονται από τις σχετικές κλίμακες των εισόδων. Επιπλέον, το StandardScaler διατηρεί τη σχετική κατανομή των δεδομένων και δεν περιορίζει αυθαίρετα τις ακραίες τιμές, όπως θα συνέβαινε με το MinMaxScaler, κάτι που είναι χρήσιμο για δεδομένα με outliers.

In [25]:
# Get columns with numbers
num_columns = df_cleaned.select_dtypes(include=['number']).columns



# Fit and transform of numerical features
scaler = StandardScaler()
df_cleaned[num_columns] = scaler.fit_transform(df_cleaned[num_columns])

print(df_cleaned[num_columns].head())

   loan_amnt  int_rate  installment  annual_inc       dti  delinq_2yrs  \
0   0.575287  0.003131     0.949595   -0.139878  0.025532    -0.378725   
1  -0.470756  0.098707    -0.266034   -0.405011  0.108257    -0.378725   
2  -0.935665 -0.546433    -0.849908   -0.489853  0.276716    -0.378725   
3  -0.586983 -0.307492    -0.435943   -0.320169  0.245990    -0.378725   
4  -0.639286  1.842975    -0.323663   -0.485866  0.279510    -0.378725   

   inq_last_6mths  mths_since_last_delinq  open_acc   pub_rec  ...  \
0        0.476036                     NaN  2.307429 -0.333925  ...   
1       -0.667268                0.088179  0.187752  1.074707  ...   
2        0.476036                     NaN  1.070951 -0.333925  ...   
3        1.619340                     NaN  1.070951 -0.333925  ...   
4        2.762645                1.588788 -0.518807 -0.333925  ...   

   total_rec_late_fee  recoveries  collection_recovery_fee  last_pymnt_amnt  \
0           -0.039338   -0.042529                -0.046

## Subquestiion 2

In [26]:
# Drop sub_grade column in order model to not know the answer
df_model = df_cleaned.drop(columns=['sub_grade'], axis=1)

# Separate features and target
x = df_model.drop(columns=['target_loan_risk'])
y = df_model['target_loan_risk']

# Split data to train/test
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.3, random_state=42, stratify=y
)

## Subquestiion 3

## Subquestiion 4

# Question 4

## Subquestiion 1

## Subquestiion 2

## Subquestiion 3

# Question 5

## Subquestiion 1

## Subquestiion 2

## Subquestiion 3

## Subquestiion 4

## Subquestiion 5